In [1]:
# QAI Orchestration Simulation for Colab
# Paste this whole script into a single Colab cell and run.

import time
import random
from dataclasses import dataclass, field
from typing import List, Dict, Any
import pandas as pd
import json
import os

random.seed(42)  # for reproducible-ish runs

# --- Utility / Logging ---
def log(msg):
    ts = time.strftime("%H:%M:%S")
    print(f"[{ts}] {msg}")

# --- Core Framework Components ---
@dataclass
class FrameworkRegistry:
    name: str
    standards: List[str]
    templates: Dict[str, Dict[str, Any]] = field(default_factory=dict)

    def register_template(self, key, template):
        self.templates[key] = template
        log(f"Registered template '{key}' in {self.name} registry.")

@dataclass
class CoreHub:
    registry: FrameworkRegistry
    event_bus: List[str] = field(default_factory=list)
    dependency_graph: Dict[str, List[str]] = field(default_factory=dict)
    state: Dict[str, str] = field(default_factory=dict)

    def publish_event(self, event):
        self.event_bus.append({"ts": time.time(), "event": event})
        log(f"CoreHub event: {event}")

    def update_state(self, key, value):
        self.state[key] = value
        log(f"CoreHub state updated: {key} = {value}")

@dataclass
class OpsAgent:
    name: str
    capabilities: List[str]

    def execute(self, task):
        log(f"OpsAgent '{self.name}' executing task: {task['id']} - {task.get('description','')}")
        # Simulate execution time based on complexity
        complexity = task.get("complexity", 1.0)
        simulated_time = max(0.05, 0.25 * complexity * (0.5 + random.random()*0.75))
        time.sleep(simulated_time)  # short sleep to simulate work
        success = random.random() > 0.02  # small chance of failure
        log(f"OpsAgent '{self.name}' completed task: {task['id']} (success={success}, dt={simulated_time:.2f}s)")
        return {"success": success, "time": simulated_time}

@dataclass
class HumanApprover:
    role: str
    name: str

    def request_approval(self, task):
        log(f"HumanApprover '{self.name}' ({self.role}) reviewing task: {task['id']} - {task.get('description','')}")
        # Simulate human decision time (faster because QAI provides summary)
        decision_latency = 0.15 + 0.15 * random.random()
        time.sleep(decision_latency)
        approved = random.random() > 0.05  # ~95% approval in this demo
        log(f"HumanApprover '{self.name}' decision for task {task['id']}: {'APPROVED' if approved else 'REJECTED'} (dt={decision_latency:.2f}s)")
        return approved, decision_latency

# --- Layer Services ---
@dataclass
class EnterpriseService:
    registry: FrameworkRegistry
    core: CoreHub

    def intake_client_order(self, client_order):
        task_id = f"ENT-{int(time.time()*1000)%100000}"
        task = {
            "id": task_id,
            "description": f"Process client order {client_order['order_id']}",
            "complexity": 1.0,
            "client_order": client_order
        }
        cap = client_order.get("capability", "qai_modernization")
        if cap in self.registry.templates:
            task["template"] = cap
            log(f"EnterpriseService mapped order to template: {cap}")
        else:
            log("EnterpriseService: no exact template, using default mapping")
            task["template"] = "default_capability"
        self.core.publish_event(f"enterprise:order_processed:{task_id}")
        return task

@dataclass
class SystemsService:
    registry: FrameworkRegistry
    core: CoreHub

    def derive_system_spec(self, enterprise_task):
        task_id = f"SYS-{int(time.time()*1000)%100000}"
        spec = {
            "id": task_id,
            "description": f"MBSE spec from {enterprise_task['id']}",
            "complexity": 1.2,
            "icd": f"ICD-{random.randint(1000,9999)}",
            "derived_from": enterprise_task["id"]
        }
        log(f"SystemsService derived system spec: {spec['id']} (ICD={spec['icd']})")
        self.core.publish_event(f"systems:spec_created:{spec['id']}")
        return spec

@dataclass
class SoftwareService:
    registry: FrameworkRegistry
    core: CoreHub
    agents: List[OpsAgent]

    def build_and_deploy(self, system_spec):
        task_id = f"SW-{int(time.time()*1000)%100000}"
        task = {"id": task_id, "description": f"Build & deploy for {system_spec['id']}", "complexity": 1.5}
        log(f"SoftwareService preparing task {task_id} from spec {system_spec['id']}")
        # Pick a capable agent (random for demo)
        agent = random.choice(self.agents)
        result = agent.execute(task)
        if result["success"]:
            artifact = {"artifact_id": f"A-{random.randint(10000,99999)}", "sbom": True, "signed": True}
            self.core.publish_event(f"software:artifact_built:{artifact['artifact_id']}")
            self.core.update_state("last_artifact", artifact["artifact_id"])
            return {"task": task, "result": result, "artifact": artifact}
        else:
            self.core.publish_event(f"software:artifact_failed:{task_id}")
            return {"task": task, "result": result, "artifact": None}

# --- Orchestrator ---
class QAIOrchestrator:
    def __init__(self, enterprise_svc, systems_svc, software_svc, hub, approvers):
        self.enterprise_svc = enterprise_svc
        self.systems_svc = systems_svc
        self.software_svc = software_svc
        self.hub = hub
        self.approvers = approvers
        self.timeline = []

    def execute_client_order(self, client_order):
        start_time = time.time()
        log("=== Starting orchestration for client order ===")
        # Step 1: Enterprise intake
        ent_task = self.enterprise_svc.intake_client_order(client_order)
        self.timeline.append(("enterprise_intake", ent_task["id"], time.time()-start_time))

        # Human approval for program kickoff
        approved, t_approve = self.approvers[0].request_approval(ent_task)
        self.timeline.append(("enterprise_approval", ent_task["id"], t_approve))
        if not approved:
            log("Order rejected at enterprise approval - aborting")
            return {"status":"rejected", "timeline": self.timeline}

        # Step 2: Systems derivation
        sys_spec = self.systems_svc.derive_system_spec(ent_task)
        self.timeline.append(("systems_spec", sys_spec["id"], time.time()-start_time))

        # Human approval for system design review
        approved_sys, t_sys_approve = self.approvers[1].request_approval(sys_spec)
        self.timeline.append(("systems_approval", sys_spec["id"], t_sys_approve))
        if not approved_sys:
            log("System design rejected - creating change request")
            # For demo, attempt one re-derive then abort
            sys_spec = self.systems_svc.derive_system_spec(ent_task)
            approved_sys2, _ = self.approvers[1].request_approval(sys_spec)
            if not approved_sys2:
                log("Repeated rejection - aborting")
                return {"status":"rejected", "timeline": self.timeline}

        # Step 3: Software build and deploy
        sw_result = self.software_svc.build_and_deploy(sys_spec)
        self.timeline.append(("software_build", sw_result["task"]["id"], sw_result["result"]["time"]))
        if not sw_result["result"]["success"]:
            log("Build failed - opening incident and aborting deployment")
            return {"status":"build_failed", "timeline": self.timeline}

        # Human approval for production deployment (security officer)
        approved_prod, t_prod = self.approvers[2].request_approval(sw_result["task"])
        self.timeline.append(("production_approval", sw_result["task"]["id"], t_prod))
        if not approved_prod:
            log("Prod deployment rejected - rollback")
            return {"status":"rejected", "timeline": self.timeline}

        # Finalize: update core hub, mark as deployed
        artifact = sw_result["artifact"]
        self.hub.update_state("deployed_artifact", artifact["artifact_id"])
        self.hub.publish_event(f"deployment:completed:{artifact['artifact_id']}")
        total_time = time.time() - start_time
        log(f"=== Orchestration complete. Total time: {total_time:.2f}s ===")
        return {"status":"deployed", "artifact": artifact, "total_time": total_time, "timeline": self.timeline}

# --- Baseline non-QAI simulation (slower) ---
def baseline_simulation():
    start = time.time()
    log("Baseline: manual intake (slow)")
    time.sleep(0.6)
    log("Baseline: system spec creation (slow)")
    time.sleep(1.2)
    log("Baseline: manual build & testing (slow)")
    time.sleep(1.5)
    log("Baseline: manual approvals and deployment (slow)")
    time.sleep(0.8)
    total = time.time() - start
    return total

# --- Setup and run ---
def run_demo():
    # Create registries and core hub
    registry = FrameworkRegistry(name="BhadaleRegistry", standards=[
        "TOGAF", "INCOSE", "IEEE SWEBOK", "CMMI L3-5", "NIST", "NASA SE"])
    registry.register_template("qai_modernization", {"desc":"QAI transformation program template", "steps":["intake","design","build","deploy"]})
    registry.register_template("default_capability", {"desc":"Default capability mapping"})

    hub = CoreHub(registry=registry)
    agents = [OpsAgent("builder-1", ["build","package"]), OpsAgent("deployer-1", ["deploy","validate"])]
    approvers = [
        HumanApprover("Program Manager", "Asha"),
        HumanApprover("Systems Lead", "Ravi"),
        HumanApprover("Security Officer", "Neha")
    ]

    enterprise_svc = EnterpriseService(registry=registry, core=hub)
    systems_svc = SystemsService(registry=registry, core=hub)
    software_svc = SoftwareService(registry=registry, core=hub, agents=agents)

    orchestrator = QAIOrchestrator(enterprise_svc, systems_svc, software_svc, hub, approvers)

    # Client order example
    client_order = {"order_id":"ORD-2025-001", "capability":"qai_modernization", "priority":"high", "client":"Acme Industries"}
    log("Running QAI-enabled orchestration simulation...")
    result = orchestrator.execute_client_order(client_order)

    # Baseline comparison
    baseline_time = baseline_simulation()
    if result.get("status") == "deployed":
        qai_time = result["total_time"]
        speedup = baseline_time / qai_time if qai_time>0 else float("inf")
        log(f"Baseline time: {baseline_time:.2f}s | QAI-enabled time: {qai_time:.2f}s | Speedup: {speedup:.2f}x")
    else:
        log("QAI orchestration did not complete successfully; baseline comparison skipped.")

    # Timeline table
    timeline = result.get("timeline", [])
    if timeline:
        df = pd.DataFrame(timeline, columns=["step","id","duration_s"])
        # Display in Colab output
        print("\nOrchestration Timeline:")
        display(df)

    # Save report
    report = {
        "client_order": client_order,
        "result": result,
        "baseline_time_s": baseline_time,
        "core_state": hub.state,
        "events": hub.event_bus
    }
    out_path = "/content/qai_orchestration_report.json"
    with open(out_path, "w") as f:
        json.dump(report, f, indent=2)
    log(f"Report saved to {out_path}")

# Run the demo in Colab
if __name__ == "__main__":
    run_demo()


[12:47:24] Registered template 'qai_modernization' in BhadaleRegistry registry.
[12:47:24] Registered template 'default_capability' in BhadaleRegistry registry.
[12:47:24] Running QAI-enabled orchestration simulation...
[12:47:24] === Starting orchestration for client order ===
[12:47:24] EnterpriseService mapped order to template: qai_modernization
[12:47:24] CoreHub event: enterprise:order_processed:ENT-44014
[12:47:24] HumanApprover 'Asha' (Program Manager) reviewing task: ENT-44014 - Process client order ORD-2025-001
[12:47:24] HumanApprover 'Asha' decision for task ENT-44014: REJECTED (dt=0.25s)
[12:47:24] Order rejected at enterprise approval - aborting
[12:47:24] Baseline: manual intake (slow)
[12:47:24] Baseline: system spec creation (slow)
[12:47:26] Baseline: manual build & testing (slow)
[12:47:27] Baseline: manual approvals and deployment (slow)
[12:47:28] QAI orchestration did not complete successfully; baseline comparison skipped.

Orchestration Timeline:


,step,id,duration_s
0,enterprise_intake,ENT-44014,0.000025
1,enterprise_approval,ENT-44014,0.245914


[12:47:28] Report saved to /content/qai_orchestration_report.json
